### Imports & Configs

In [1]:
!pip install groq
import time
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [25]:
from groq import Groq
from google.colab import userdata

groq_api_key = userdata.get('groq_api_2')
groq_client = Groq(api_key=groq_api_key)

In [4]:
llama8b = "llama3-8b-8192"

In [5]:
def ask_llm(prompts, client, model):
    responses = []
    for prompt in prompts:
      response = client.chat.completions.create(
          model=model,
          messages=[
              {"role": "user", "content": prompt[0]}
          ],
          max_tokens=2000
      )

      responses.append({
          "user_prompt" : prompt[0],
          "response" : response.choices[0].message.content,
          "post_tone": prompt[1]['tone'],
          "post_type": prompt[1]['post_type'],
          "post_length": prompt[1]['length']
      })

    return responses

###Synthentic Dataset Configs

In [6]:
universal_criteria = f"""
- Capture attention quickly with bold statements or questions.
- Keep the text concise and to the point.
- Use a strong, clear call-to-action (CTA).
- Highlight the benefits of the product.
- Emphasize unique selling points (USPs).
- Maintain consistent branding elements.
- Leverage platform-specific algorithms (e.g., hashtags, keywords).
"""


In [7]:
short_posts = [
    {
        "type": "Promotional Posts",
        "length": "1-3 sentences",
        "criteria": f"""
        - Announce sales, discounts, or special offers.
        - Highlight urgency with time-sensitive language.
        - Mention exclusivity in offers.
        """
    },
    {
        "type": "Question Posts",
        "length": "1-3 sentences",
        "criteria": f"""
        - Engage the audience with questions related to the product.
        - Personalize messages to the audience.
        """
    },
    {
        "type": "Feature Highlight Posts",
        "length": "1-3 sentences",
        "criteria": f"""
        - Focus on specific features or benefits of the product.
        - Use vivid, descriptive language.
        """
    },
    {
        "type": "Comparison Posts",
        "length": "1-3 sentences",
        "criteria": f"""
        - Compare the product with competitors or previous versions.
        """
    },
    {
        "type": "Announcement Posts",
        "length": "1-3 sentences",
        "criteria": f"""
        - Inform about new product launches or updates.
        - Align posts with seasonal or trendy themes.
        """
    }
]

In [8]:
medium_posts = [
    {
        "type": "Educational Posts",
        "length": "4-6 sentences",
        "criteria": f"""
        - Provide useful information or tips related to the product.
        - Share educational snippets or interesting facts.
        """
    },
    {
        "type": "Testimonial Posts",
        "length": "4-6 sentences",
        "criteria": f"""
        - Share customer reviews and testimonials.
        - Quote satisfied customers or share positive reviews.
        - Include social proof elements like customer numbers or awards.
        """
    },
    {
        "type": "User-Generated Content Posts",
        "length": "4-6 sentences",
        "criteria": f"""
        - Share content created by customers.
        - Highlight user experiences and benefits from the product.
        """
    },
    {
        "type": "Value Proposition Posts",
        "length": "4-6 sentences",
        "criteria": f"""
        - Clearly state what sets the product apart from others.
        - Use a problem-solution framework.
        """
    }
]

In [9]:
long_posts = [
    {
        "type": "Storytelling Posts",
        "length": "7+ sentences",
        "criteria": f"""
        - Narrate a story that highlights the product’s benefits.
        - Use vivid, descriptive language.
        - Personalize the story to the audience.
        """
    },
    {
        "type": "Interactive Posts",
        "length": "7+ sentences",
        "criteria": f"""
        - Include polls, quizzes, or contests to engage the audience.
        - Use interactive elements to increase engagement.
        - Encourage audience participation and sharing.
        """
    }
]

In [10]:
companies = [
    {
        "company_name": "EcoTech Solutions",
        "company_description": f"""
        Industry: Environmental Technology
        Mission Statement: To provide innovative and sustainable technology solutions that protect and enhance the environment.
        Unique Selling Proposition (USP): Leading provider of eco-friendly technologies with a strong focus on reducing carbon footprint.
        Target Market: Businesses and government agencies seeking sustainable technology solutions.
        Product Offering: Advanced filtration systems, renewable energy solutions, eco-friendly building materials, waste management technology, and sustainable agriculture tools.
        Company Size: Mid-sized with a global presence, employing over 500 professionals.
        History: Founded in 2005, EcoTech Solutions has been at the forefront of environmental innovation, consistently delivering cutting-edge products that meet the highest standards of sustainability.
        Target Audience: Environmentally conscious businesses, government agencies, and non-profit organizations focused on sustainability and reducing environmental impact.
        """,
        "products": [
            "EcoClean Water Filtration System: Advanced filtration system for purifying water in homes and businesses.",
            "SolarMax Photovoltaic Panels: High-efficiency solar panels for generating renewable energy.",
            "GreenBuild Insulation Materials: Eco-friendly insulation materials for energy-efficient buildings.",
            "WasteAway Recycling Units: Innovative recycling units for managing waste sustainably.",
            "AgriTech Drip Irrigation System: Efficient irrigation system designed to conserve water in agriculture.",
            "WindGen Turbines: Wind turbines for harnessing wind energy.",
            "RainHarvest Collection Systems: Systems for collecting and utilizing rainwater.",
            "BioPack Biodegradable Packaging: Environmentally friendly packaging made from biodegradable materials.",
            "EVCharge Pro Stations: Electric vehicle charging stations for public and private use.",
            "HydroFlow Micro-Hydro Generators: Small-scale hydro generators for generating electricity from water flow.",
            "GeoTherm Heating Systems: Geothermal heating systems for sustainable temperature control.",
            "EcoSmart Thermostats: Smart thermostats that optimize energy usage.",
            "SolarRoof Shingles: Solar energy-generating roof shingles.",
            "GreenLight LED Lighting: Energy-efficient LED lighting solutions.",
            "EcoVentilation Systems: Ventilation systems that improve air quality and energy efficiency.",
            "EnviroHeat Pumps: Heat pumps that provide efficient heating and cooling.",
            "HydroPower Generators: Generators that produce electricity from hydro sources.",
            "EcoConserve Water Savers: Devices that reduce water consumption.",
            "BioWaste Digesters: Systems that convert organic waste into useful byproducts.",
            "SmartGrid Energy Management: Solutions for managing and optimizing energy distribution."
        ]
    },
    {
        "company_name": "HealthPro Pharmaceuticals",
        "company_description": f"""
        Industry: Pharmaceuticals
        Mission Statement: To develop and distribute high-quality pharmaceutical products that improve patient health and quality of life.
        Unique Selling Proposition (USP): Commitment to research and development, ensuring the highest efficacy and safety standards for all products.
        Target Market: Healthcare providers, hospitals, and individual consumers.
        Product Offering: Prescription medications, over-the-counter drugs, vaccines, medical devices, and nutritional supplements.
        Company Size: Large multinational corporation with over 10,000 employees worldwide.
        History: Established in 1980, HealthPro Pharmaceuticals has a long-standing reputation for innovation and excellence in the pharmaceutical industry.
        Target Audience: Healthcare professionals, hospitals, and patients seeking reliable and effective medical treatments.
        """,
        "products": [
            "CardioPro Blood Pressure Medication: Medication to manage high blood pressure.",
            "HealthPro Pain Relief Gel: Topical gel for relieving pain.",
            "ImmuneBoost Flu Vaccine: Vaccine to prevent influenza.",
            "GlucoCheck Blood Glucose Monitor: Device for monitoring blood glucose levels.",
            "NutriLife Multivitamins: Daily multivitamin supplement.",
            "Respira Inhaler: Inhaler for respiratory relief.",
            "NeuroPlus Cognitive Enhancer: Supplement to improve cognitive function.",
            "AllerFree Allergy Medication: Medication to relieve allergy symptoms.",
            "DiabeCare Insulin Pump: Insulin pump for diabetes management.",
            "ThermoPatch Heat Patches: Heat patches for pain relief.",
            "VitaBoost Vitamin D: Vitamin D supplement for bone health.",
            "OsteoGuard Bone Health: Supplement to support bone density.",
            "CoughCare Syrup: Syrup for cough relief.",
            "Dermacare Skin Ointment: Ointment for skin conditions.",
            "SleepWell Tablets: Tablets to aid sleep.",
            "HeartStrong Omega-3: Omega-3 supplement for heart health.",
            "FlexiJoint Glucosamine: Supplement for joint health.",
            "CalmEase Anti-Anxiety: Medication to reduce anxiety.",
            "FocusSharp Nootropic: Supplement to enhance mental focus.",
            "LiverHealth Detox: Supplement to support liver detoxification."
        ]
    },
    {
        "company_name": "TechNova Innovations",
        "company_description": f"""
        Industry: Consumer Electronics
        Mission Statement: To create cutting-edge consumer electronics that enhance everyday life through technology.
        Unique Selling Proposition (USP): Pioneering innovations in smart technology and user-friendly designs.
        Target Market: Tech-savvy consumers and businesses looking for the latest in electronic gadgets.
        Product Offering: Smart home devices, wearable technology, personal gadgets, high-performance laptops, and smart accessories.
        Company Size: Growing company with a workforce of 2,000 employees and a significant market share in North America and Europe.
        History: Founded in 2010, TechNova Innovations has quickly become a leader in the consumer electronics market by consistently pushing the boundaries of technology.
        Target Audience: Tech enthusiasts, early adopters, and professionals seeking high-performance and innovative electronic devices.
        """,
        "products": [
            "HomeGuard Smart Security System: Advanced security system for smart homes.",
            "FitTrack Health Wearable: Wearable device for tracking fitness and health metrics.",
            "NovaTab Pro Tablet: High-performance tablet for work and entertainment.",
            "PowerBook Ultra Laptop: Powerful laptop for professional use.",
            "EchoBuds Wireless Earbuds: Wireless earbuds with superior sound quality.",
            "VistaVR Headset: Virtual reality headset for immersive experiences.",
            "SyncWatch Smartwatch: Smartwatch with fitness tracking and notifications.",
            "AuraSmart Light Bulbs: Smart light bulbs with adjustable brightness and colors.",
            "ChargeFast Wireless Charger: Fast wireless charger for multiple devices.",
            "SoundWave Bluetooth Speakers: High-quality Bluetooth speakers for home and travel.",
            "NanoScreen Protector: Durable screen protector for smartphones and tablets.",
            "AirPure Smart Air Purifier: Air purifier with smart controls and HEPA filters.",
            "SmartLock Door System: Smart locking system for enhanced home security.",
            "ThermoSense Smart Thermostat: Smart thermostat that learns and adapts to your schedule.",
            "RoboClean Vacuum: Robot vacuum cleaner with smart navigation.",
            "WaterSense Smart Faucet: Smart faucet that conserves water.",
            "CamGuard Security Camera: High-definition security camera with remote access.",
            "PowerHub Multi-Device Charger: Multi-device charger for simultaneous charging.",
            "EchoBeam Projector: High-definition projector for home theaters.",
            "SmartShade Automated Blinds: Automated blinds that adjust based on sunlight."
        ]
    },
    {
        "company_name": "PureBeauty Cosmetics",
        "company_description": f"""
        Industry: Cosmetics and Skincare
        Mission Statement: To provide high-quality, natural, and eco-friendly beauty products that enhance the natural beauty of every individual.
        Unique Selling Proposition (USP): Use of natural ingredients and sustainable packaging.
        Target Market: Environmentally conscious consumers looking for natural beauty solutions.
        Product Offering: Natural skincare products, organic makeup, eco-friendly hair care, fragrance-free lotions, and anti-aging serums.
        Company Size: Small to mid-sized company with 500 employees and a growing presence in the online beauty market.
        History: Since its founding in 2015, PureBeauty Cosmetics has been dedicated to promoting natural beauty and sustainability in the cosmetics industry.
        Target Audience: Eco-conscious consumers, individuals seeking natural and organic beauty products, and customers interested in sustainable living.
        """,
        "products": [
            "RadiantGlow Face Cream: Face cream for radiant and hydrated skin.",
            "EcoLuxe Foundation: Eco-friendly foundation with natural ingredients.",
            "HerbalEssence Shampoo: Herbal shampoo for healthy hair.",
            "ClearSkin Acne Treatment: Acne treatment for clear skin.",
            "YouthRenew Anti-Aging Serum: Anti-aging serum for youthful skin.",
            "NaturalBalm Lip Care: Lip balm with natural ingredients.",
            "PureProtect Sunscreen: Sunscreen with broad-spectrum protection.",
            "Botanical Bliss Body Wash: Body wash with botanical extracts.",
            "SilkTouch Hair Conditioner: Hair conditioner for silky smooth hair.",
            "NatureFresh Deodorant: Natural deodorant without harmful chemicals.",
            "GreenTea Facial Cleanser: Facial cleanser with green tea extracts.",
            "VitaminC Brightening Serum: Serum with Vitamin C for brightening skin.",
            "HydraBoost Moisturizer: Moisturizer for intense hydration.",
            "SoothingAloe Gel: Aloe gel for soothing skin.",
            "Lavender Night Cream: Night cream with lavender for relaxation.",
            "RevitalEyes Eye Cream: Eye cream for reducing dark circles and puffiness.",
            "MineralRich Mud Mask: Mud mask with rich minerals for detoxifying skin.",
            "Organic Rosewater Toner: Toner with organic rosewater for refreshing skin.",
            "Chamomile Calming Mist: Calming mist with chamomile for soothing skin.",
            "Peppermint Foot Cream: Foot cream with peppermint for refreshing feet."
        ]
    },
    {
        "company_name": "EduFuture Learning",
        "company_description": f"""
        Industry: Education Technology
        Mission Statement: To revolutionize education through technology, making learning accessible and engaging for everyone.
        Unique Selling Proposition (USP): Innovative e-learning platforms and interactive educational tools.
        Target Market: Students, educators, and educational institutions.
        Product Offering: E-learning platforms, interactive educational tools, virtual classrooms, educational software, and digital textbooks.
        Company Size: Mid-sized company with 1,200 employees and a strong presence in the educational technology sector.
        History: Founded in 2012, EduFuture Learning has been a pioneer in integrating technology with education, offering a wide range of products that cater to modern learning needs.
        Target Audience: Students, teachers, schools, and educational institutions looking to integrate technology into their teaching and learning processes.
        """,
        "products": [
            "LearnX E-learning Platform: Comprehensive platform for online learning.",
            "EduInteractive Whiteboard: Interactive whiteboard for engaging lessons.",
            "VirtualClass Pro: Virtual classroom software for remote teaching.",
            "MathMaster Educational Software: Software for mastering mathematics.",
            "E-Textbook Library: Digital library of textbooks for various subjects.",
            "QuizMaster Online Assessment: Online assessment tool for quizzes and exams.",
            "LangLearn Language App: Language learning app with interactive lessons.",
            "STEMExplorer Kit: Kit for exploring STEM concepts hands-on.",
            "VirtualLab Science Simulations: Simulations for virtual science experiments.",
            "AdaptiveLearn System: Adaptive learning system that personalizes education.",
            "HistoryHaven Digital Archive: Digital archive of historical documents and artifacts.",
            "GeoExplore Virtual Globe: Virtual globe for geography lessons.",
            "ChemLab Interactive Experiments: Interactive experiments for chemistry students.",
            "BioWorld Virtual Biology Lab: Virtual lab for biology experiments.",
            "PhysiCalc Physics Simulator: Physics simulator for understanding physical concepts.",
            "LiteratureLink Annotated Texts: Annotated texts for literature studies.",
            "ArtStudio Digital Art Tools: Digital tools for creating and studying art.",
            "MusicMaestro Learning Platform: Platform for learning and practicing music.",
            "EconEdu Financial Literacy Program: Program for teaching financial literacy.",
            "CoderPro Programming Courses: Courses for learning various programming languages."
        ]
    },
    {
        "company_name": "GreenHomes Real Estate",
        "company_description": f"""
        Industry: Real Estate
        Mission Statement: To create sustainable, eco-friendly homes that provide a comfortable living environment while protecting the planet.
        Unique Selling Proposition (USP): Specializes in energy-efficient, eco-friendly homes with modern amenities.
        Target Market: Environmentally conscious homebuyers looking for sustainable living solutions.
        Product Offering: Energy-efficient homes, sustainable community developments, eco-friendly commercial properties, green building consultations, and solar-powered residential areas.
        Company Size: Mid-sized company with 700 employees and projects across the country.
        History: Founded in 2010, GreenHomes Real Estate has been dedicated to providing sustainable living solutions, setting new standards in eco-friendly construction.
        Target Audience: Environmentally conscious homebuyers, investors in sustainable real estate, and families seeking eco-friendly living environments.
        """,
        "products": [
            "SolarVille Housing Project: Residential project with solar-powered homes.",
            "EcoPark Community Development: Sustainable community development.",
            "GreenTower Commercial Complex: Eco-friendly commercial office spaces.",
            "Sunshine Apartments: Energy-efficient apartment buildings.",
            "NatureHaven Villas: Villas designed with natural and sustainable materials.",
            "EcoLiving Townhouses: Townhouses with green building practices.",
            "GreenCity Condominiums: High-rise condominiums with sustainable features.",
            "Rainforest Estates: Luxury estates surrounded by preserved nature.",
            "EcoVillage Residential Area: Community with eco-friendly homes and amenities.",
            "SolarHeights Office Building: Office building powered by solar energy.",
            "ForestView Residences: Residences with views of forested areas.",
            "WindFarm Estates: Estates powered by wind energy.",
            "EcoLoft Urban Living: Urban living spaces with eco-friendly design.",
            "GreenBelt Communities: Communities with green spaces and sustainable housing.",
            "HydroHome Developments: Homes designed with water conservation features.",
            "BioDome Residences: Residences within biodome environments.",
            "SunBloom Gardens: Garden-style residences with sustainable features.",
            "RenewWave Apartments: Apartments with renewable energy systems.",
            "GeoLiving Complex: Complex designed with geothermal energy solutions.",
            "PureAir Estates: Estates with advanced air purification systems."
        ]
    },
    {
        "company_name": "FitLife Apparel",
        "company_description": f"""
        Industry: Sportswear and Fitness Apparel
        Mission Statement: To provide high-quality, comfortable, and stylish fitness apparel that supports an active lifestyle.
        Unique Selling Proposition (USP): Combines advanced fabric technology with trendy designs for optimal performance and style.
        Target Market: Fitness enthusiasts, athletes, and active individuals.
        Product Offering: Performance sportswear, fitness accessories, activewear for women, men’s workout gear, and eco-friendly yoga clothing.
        Company Size: Mid-sized company with 600 employees and a strong online presence.
        History: Founded in 2012, FitLife Apparel has grown to become a trusted brand in the fitness industry, known for its innovative designs and high-quality products.
        Target Audience: Fitness enthusiasts, athletes, and active individuals looking for stylish and high-performance workout clothing.
        """,
        "products": [
            "FlexFit Leggings: Leggings designed for maximum flexibility and comfort.",
            "PowerCore Compression Shorts: Compression shorts for enhanced athletic performance.",
            "EcoYoga Mats: Eco-friendly yoga mats.",
            "RunFast Running Shoes: Running shoes designed for speed and comfort.",
            "LiftStrong Weightlifting Gloves: Gloves for safe and effective weightlifting.",
            "CoolDry T-Shirts: T-shirts with moisture-wicking technology.",
            "ThermoTrack Hoodies: Hoodies designed for temperature regulation.",
            "BalanceFit Sports Bras: Sports bras with excellent support.",
            "ProStretch Yoga Pants: Yoga pants with optimal stretch.",
            "Endurance Socks: Socks designed for endurance activities.",
            "QuickDry Swimwear: Swimwear that dries quickly.",
            "ReflectiveJog Safety Gear: Safety gear with reflective elements for night jogging.",
            "ActiveFlex Tracksuits: Tracksuits designed for active lifestyles.",
            "UltraLight Jackets: Lightweight jackets for outdoor activities.",
            "CoreStrength Athletic Belts: Athletic belts for core support.",
            "SpeedGrip Training Shoes: Training shoes with superior grip.",
            "StretchTech Resistance Bands: Resistance bands for strength training.",
            "HydroFlow Water Bottles: Water bottles designed for hydration on the go.",
            "PulseMonitor Fitness Tracker: Fitness tracker for monitoring vital signs.",
            "ZenBalance Meditation Cushions: Cushions designed for meditation practice."
        ]
    },
    {
        "company_name": "Gourmet Delights",
        "company_description": f"""
        Industry: Food and Beverage
        Mission Statement: To provide premium, gourmet food products that deliver exceptional taste and quality.
        Unique Selling Proposition (USP): Offers a wide range of high-quality, artisanal food products made with the finest ingredients.
        Target Market: Food enthusiasts, gourmet chefs, and consumers seeking premium food products.
        Product Offering: Artisanal cheeses, gourmet chocolates, premium olive oils, specialty spices, and organic snacks.
        Company Size: Small company with 150 employees and a growing online market presence.
        History: Established in 2015, Gourmet Delights has quickly gained a reputation for its high-quality products and commitment to excellence.
        Target Audience: Food enthusiasts, gourmet chefs, and discerning consumers looking for high-quality, artisanal food products.
        """,
        "products": [
            "Truffle Infused Olive Oil: Olive oil infused with rich truffle flavor.",
            "Gourmet Dark Chocolate Bars: High-quality dark chocolate bars.",
            "Artisanal Cheddar Cheese: Handcrafted cheddar cheese.",
            "Organic Spicy Nuts: Organic nuts with a spicy twist.",
            "Premium Vanilla Extract: High-quality vanilla extract.",
            "Heritage Breed Pork Sausages: Sausages made from heritage breed pork.",
            "Handmade Belgian Waffles: Traditional Belgian waffles.",
            "Wildflower Honey: Pure honey from wildflowers.",
            "Balsamic Vinegar of Modena: Authentic balsamic vinegar.",
            "Mediterranean Sea Salt: Sea salt from the Mediterranean.",
            "French Brie Cheese: Creamy French brie cheese.",
            "Italian Espresso Beans: Premium Italian espresso beans.",
            "Maple Glazed Salmon: Salmon glazed with maple syrup.",
            "Black Truffle Butter: Butter infused with black truffle.",
            "Gourmet Popcorn Variety: Assorted gourmet popcorn flavors.",
            "Smoked Almonds: Almonds with a smoky flavor.",
            "Artisan Sourdough Bread: Handcrafted sourdough bread.",
            "Spiced Chai Tea: Chai tea with a blend of spices.",
            "Exotic Fruit Jam: Jam made from exotic fruits.",
            "Organic Pasta Selection: Assorted organic pasta varieties."
        ]
    },
    {
        "company_name": "GreenBytes Tech",
        "company_description": f"""
        Industry: Software Development
        Mission Statement: To develop innovative software solutions that promote environmental sustainability and efficiency.
        Unique Selling Proposition (USP): Leading provider of software solutions focused on optimizing energy use and reducing environmental impact.
        Target Market: Businesses and organizations committed to sustainability.
        Product Offering: Energy management software, sustainability reporting tools, carbon footprint calculators, green supply chain management software, and eco-friendly project management tools.
        Company Size: Mid-sized company with 300 employees and a growing global presence.
        History: Founded in 2010, GreenBytes Tech has been dedicated to helping organizations achieve their sustainability goals through innovative software solutions.
        Target Audience: Businesses and organizations looking to optimize energy use, reduce carbon footprint, and achieve sustainability goals.
        """,
      "products": [
          "EcoManage Energy Management Software: Software for managing energy usage.",
          "SustainReport Sustainability Reporting Tools: Tools for sustainability reporting.",
          "CarbonCalc Footprint Calculator: Calculator for measuring carbon footprint.",
          "GreenChain Supply Management: Software for green supply chain management.",
          "EcoProject Project Management Tool: Project management software for eco-projects.",
          "ResourceSaver Efficiency Suite: Suite of tools for resource efficiency.",
          "GreenAudit Compliance Checker: Software for checking environmental compliance.",
          "EnviroTracker Impact Monitoring: Monitoring tool for environmental impact.",
          "EcoPlan Strategic Planning Software: Software for strategic environmental planning.",
          "SustainDash Analytics Dashboard: Dashboard for sustainability analytics.",
          "EnergyWatch Consumption Monitor: Monitor for tracking energy consumption.",
          "RenewTrack Renewable Integration: Tool for integrating renewable energy.",
          "WasteReduce Waste Management: Software for managing waste reduction.",
          "CarbonMap Emission Mapper: Mapping tool for carbon emissions.",
          "EcoFleet Fleet Management: Management tool for eco-friendly fleets.",
          "WaterWise Usage Tracker: Tracker for water usage.",
          "GreenBudget Financial Planning: Financial planning tool for green projects.",
          "RecycleSmart Waste Optimization: Tool for optimizing waste recycling.",
          "EcoScore Performance Metrics: Metrics tool for environmental performance.",
          "EnviroSync Integration Platform: Platform for integrating environmental data."
      ]
    },
    {
        "company_name": "PureSound Audio",
        "company_description": f"""
        Industry: Audio Equipment
        Mission Statement: To deliver the highest quality audio experience through innovative and precision-engineered products.
        Unique Selling Proposition (USP): Combines state-of-the-art technology with premium materials to produce superior sound quality.
        Target Market: Audiophiles, musicians, and professional audio engineers.
        Product Offering: High-fidelity speakers, noise-cancelling headphones, studio monitors, portable Bluetooth speakers, and high-resolution audio players.
        Company Size: Mid-sized company with 400 employees and a strong presence in both consumer and professional audio markets.
        History: Founded in 2012, PureSound Audio has rapidly become a respected name in the audio equipment industry, known for its commitment to quality and innovation.
        Target Audience: Audiophiles, musicians, and professional audio engineers seeking high-quality, precision-engineered audio equipment.
        """,
        "products": [
            "PureBass High-Fidelity Speakers: Speakers with high-fidelity sound quality.",
            "NoiseShield Pro Headphones: Headphones with advanced noise-cancelling technology.",
            "StudioMaster Monitors: Studio monitors for professional audio production.",
            "PocketSound Bluetooth Speaker: Portable Bluetooth speaker.",
            "HiRes Audio Player: High-resolution audio player.",
            "ClearVoice Microphone: Microphone with clear sound capture.",
            "BassBoost Subwoofer: Subwoofer with enhanced bass.",
            "WirelessSound Home System: Wireless home audio system.",
            "SoundSync Wireless Transmitter: Wireless transmitter for audio devices.",
            "PureTone Earbuds: Earbuds with pure sound quality.",
            "AcousticPro Sound Panels: Sound panels for acoustic treatment.",
            "ConcertMaster Amplifier: Amplifier for live performances.",
            "StudioConnect Audio Interface: Audio interface for recording studios.",
            "SoundBoost Earplugs: Earplugs for sound protection.",
            "PureBass Wireless Earbuds: Wireless earbuds with pure bass.",
            "NoiseCancel Over-Ear Headphones: Over-ear headphones with noise-cancelling technology.",
            "LiveStream Microphone: Microphone for live streaming.",
            "HomeTheater Surround System: Surround sound system for home theaters.",
            "ProMix Audio Mixer: Audio mixer for professional use.",
            "QuietComfort Noise Cancelling Headphones: Comfortable headphones with noise-cancelling features."
        ]
    },
    {
        "company_name": "HomeHaven Furnishings",
        "company_description": f"""
        Industry: Home Furnishings
        Mission Statement: To provide stylish, comfortable, and affordable home furnishings that enhance everyday living.
        Unique Selling Proposition (USP): Offers a wide range of modern and classic furniture designs made from high-quality materials.
        Target Market: Homeowners, interior designers, and renters looking for stylish and functional furniture.
        Product Offering: Living room furniture, bedroom sets, dining room tables, office furniture, and home decor accessories.
        Company Size: Mid-sized company with 500 employees and a growing online and retail presence.
        History: Founded in 2013, HomeHaven Furnishings has become a trusted name in home furnishings, known for its stylish designs and affordable prices.
        Target Audience: Homeowners, interior designers, and renters seeking stylish, comfortable, and affordable home furnishings.
        """,
        "products": [
            "ComfortPlus Sofa: Sofa designed for maximum comfort.",
            "Elegance Dining Table: Elegant dining table for stylish dining rooms.",
            "Modern Loft Bed Frame: Bed frame with a modern loft design.",
            "OfficePro Desk: Desk designed for professional use.",
            "Luxury Recliner Chair: Recliner chair with luxurious comfort.",
            "Urban Chic Coffee Table: Stylish coffee table for urban homes.",
            "Classic Wardrobe: Classic design wardrobe for ample storage.",
            "Designer Lamp Collection: Collection of designer lamps.",
            "EcoWood Bookshelves: Bookshelves made from eco-friendly wood.",
            "Premium Mattress Set: High-quality mattress set for ultimate comfort.",
            "VelvetLuxe Armchair: Velvet armchair for a luxurious feel.",
            "SmartHome Entertainment Unit: Entertainment unit with smart features.",
            "RusticKitchen Island: Rustic-style kitchen island.",
            "Vintage Mirror Set: Set of vintage mirrors.",
            "SpaceSaver Shoe Rack: Shoe rack designed to save space.",
            "DualComfort Mattress: Mattress with dual comfort zones.",
            "Heritage Dining Set: Traditional dining set with heritage design.",
            "RelaxZone Bean Bag: Comfortable bean bag chair.",
            "Craftsman Workbench: Workbench designed for craftsmen.",
            "Timeless Bedside Table: Classic bedside table design."
        ]
    },
    {
        "company_name": "BioLife Supplements",
        "company_description": f"""
        Industry: Health and Wellness
        Mission Statement: To promote health and wellness by providing high-quality, natural supplements that support a healthy lifestyle.
        Unique Selling Proposition (USP): Uses only the highest quality natural ingredients to create effective health supplements.
        Target Market: Health-conscious individuals, fitness enthusiasts, and wellness seekers.
        Product Offering: Dietary supplements, vitamins and minerals, herbal extracts, protein powders, and probiotics.
        Company Size: Small to mid-sized company with 200 employees and a growing customer base.
        History: Founded in 2015, BioLife Supplements has quickly established itself as a trusted provider of natural health supplements.
        Target Audience: Health-conscious individuals, fitness enthusiasts, and wellness seekers looking for natural and effective supplements.
        """,
        "products": [
            "BioLife Multivitamin: Daily multivitamin supplement.",
            "ProBio Advanced Probiotics: Probiotics for digestive health.",
            "HerbalBoost Energy Capsules: Energy capsules with herbal ingredients.",
            "VitaC Immune Support: Vitamin C supplement for immune support.",
            "PureOmega Fish Oil: Fish oil supplement for heart health.",
            "PlantPower Protein Powder: Plant-based protein powder.",
            "JointFlex Glucosamine: Glucosamine supplement for joint health.",
            "DigestEase Enzyme Blend: Enzyme blend for digestive health.",
            "CalMag Calcium Magnesium: Calcium and magnesium supplement.",
            "SleepWell Melatonin: Melatonin supplement for sleep support.",
            "LiverCare Detox: Detox supplement for liver health.",
            "VisionGuard Lutein: Lutein supplement for eye health.",
            "HeartHealth CoQ10: CoQ10 supplement for heart health.",
            "StressRelief Ashwagandha: Ashwagandha supplement for stress relief.",
            "SkinGlow Collagen: Collagen supplement for skin health.",
            "MemorySharp Ginkgo Biloba: Ginkgo Biloba supplement for cognitive health.",
            "MuscleMax BCAA: BCAA supplement for muscle recovery.",
            "EnergyPlus Iron Supplement: Iron supplement for energy support.",
            "BoneStrength Calcium: Calcium supplement for bone health.",
            "AntiInflame Turmeric: Turmeric supplement for anti-inflammatory support."
        ]
    },
    {
        "company_name": "UrbanEats Food Truck",
        "company_description": f"""
        Industry: Food and Beverage
        Mission Statement: To deliver delicious, gourmet street food made with fresh, locally sourced ingredients.
        Unique Selling Proposition (USP): Combines gourmet flavors with the convenience of street food.
        Target Market: Urban dwellers, foodies, and busy professionals looking for high-quality, convenient meals.
        Product Offering: Gourmet burgers, artisanal sandwiches, fresh salads, specialty tacos, and craft beverages.
        Company Size: Small company with 50 employees and a fleet of food trucks operating in major cities.
        History: Established in 2018, UrbanEats Food Truck has quickly become a favorite among city dwellers for its delicious and convenient gourmet street food.
        Target Audience: Urban dwellers, foodies, and busy professionals seeking high-quality, convenient meals.
        """,
        "products": [
            "Gourmet Angus Burger: High-quality Angus beef burger.",
            "Truffle Grilled Cheese: Grilled cheese sandwich with truffle flavor.",
            "Quinoa Power Salad: Salad with quinoa and fresh vegetables.",
            "BBQ Pulled Pork Sandwich: Sandwich with BBQ pulled pork.",
            "Spicy Fish Tacos: Tacos with spicy fish filling.",
            "Veggie Delight Wrap: Wrap with assorted vegetables.",
            "Lemonade Fresh: Freshly made lemonade.",
            "Craft Root Beer: Craft-brewed root beer.",
            "Loaded Nachos: Nachos loaded with toppings.",
            "Street Corn Elote: Mexican-style street corn.",
            "Buffalo Chicken Wings: Chicken wings with buffalo sauce.",
            "Gourmet Mac and Cheese: Mac and cheese with gourmet ingredients.",
            "Chipotle Steak Burrito: Burrito with chipotle steak.",
            "Hawaiian Poke Bowl: Poke bowl with Hawaiian flavors.",
            "Sweet Potato Fries: Fries made from sweet potatoes.",
            "Falafel Wrap: Wrap with falafel and fresh vegetables.",
            "Artisan Pizza Slices: Slices of artisan pizza.",
            "Hand-Crafted Ice Cream: Handmade ice cream.",
            "Sriracha Chicken Sandwich: Chicken sandwich with sriracha sauce.",
            "Grilled Veggie Skewers: Skewers with grilled vegetables."
        ]
    },
    {
        "company_name": "AdventureGear Co.",
        "company_description": f"""
        Industry: Outdoor Equipment
        Mission Statement: To provide high-quality, durable outdoor gear that enhances the adventure experience.
        Unique Selling Proposition (USP): Combines rugged durability with innovative design to create gear that performs in the toughest conditions.
        Target Market: Outdoor enthusiasts, adventure travelers, and athletes.
        Product Offering: Hiking gear, camping equipment, climbing gear, water sports equipment, and adventure accessories.
        Company Size: Mid-sized company with 300 employees and a global customer base.
        History: Founded in 2010, AdventureGear Co. has become a trusted name in outdoor equipment, known for its durable and innovative products.
        Target Audience: Outdoor enthusiasts, adventure travelers, and athletes looking for high-quality, durable gear for their adventures.
        """,
        "products": [
            "TrekMaster Hiking Boots: Durable hiking boots for all terrains.",
            "EcoTent Camping Tent: Eco-friendly camping tent.",
            "ClimbSafe Harness: Safety harness for climbing.",
            "AquaPro Kayak: Professional-grade kayak.",
            "TrailBlaze Backpack: Backpack designed for hiking.",
            "ExtremeWeather Sleeping Bag: Sleeping bag for extreme weather conditions.",
            "MountainPeak Trekking Poles: Trekking poles for mountain hiking.",
            "RapidInflate Sleeping Pad: Inflatable sleeping pad.",
            "HydroFlow Water Filter: Water filter for outdoor use.",
            "Survivor Multi-Tool: Multi-tool with various functions.",
            "SolarCharge Portable Panel: Portable solar panel for charging devices.",
            "WindResist Jacket: Jacket designed to resist wind.",
            "FireStarter Kit: Kit for starting fires.",
            "GeoCompass Navigation Device: Navigation device for geolocation.",
            "RockClimber Chalk Bag: Bag for climbing chalk.",
            "StormGuard Rain Gear: Rain gear for protection against storms.",
            "TrailMaster GPS: GPS device for trail navigation.",
            "CampCook Portable Stove: Portable stove for camping.",
            "ThermoPack Insulated Bottle: Insulated bottle for keeping drinks hot or cold.",
            "DrySafe Waterproof Bag: Waterproof bag for protecting gear."
        ]
    },
    {
        "company_name": "TechHealth Solutions",
        "company_description": f"""
        Industry: Health Technology
        Mission Statement: To improve healthcare outcomes through innovative technology solutions.
        Unique Selling Proposition (USP): Provides advanced technology solutions that streamline healthcare processes and improve patient care.
        Target Market: Hospitals, clinics, and healthcare providers.
        Product Offering: Health management software, telehealth solutions, patient monitoring systems, electronic health records, and medical data analytics.
        Company Size: Mid-sized company with 500 employees and a strong presence in the healthcare technology market.
        History: Founded in 2012, TechHealth Solutions has been dedicated to revolutionizing healthcare through technology, offering a range of innovative products.
        Target Audience: Hospitals, clinics, and healthcare providers seeking advanced technology solutions to improve patient care and streamline operations.
        """,
        "products": [
            "HealthManage Software: Software for managing healthcare services.",
            "TeleCare Telehealth Platform: Platform for telehealth services.",
            "VitalsMonitor Patient System: Patient monitoring system.",
            "EHRPlus Electronic Health Records: Electronic health records system.",
            "MediData Analytics: Analytics tool for medical data.",
            "CareConnect Communication System: Communication system for healthcare providers.",
            "RemoteTrack Monitoring Devices: Devices for remote patient monitoring.",
            "SmartAlert Emergency Response: Emergency response system.",
            "HealthPortal Patient Interface: Interface for patient engagement.",
            "BioScan Diagnostic Tools: Diagnostic tools for healthcare professionals.",
            "MediTrack Wearable: Wearable device for tracking health metrics.",
            "DocLink Physician Network: Network for physician collaboration.",
            "ClinicFlow Management: Management software for clinics.",
            "PatientConnect App: App for patient communication and management.",
            "HealthSync Integration: Integration platform for health systems.",
            "PharmaCheck Drug Monitoring: Drug monitoring system.",
            "SurgiPlan Pre-Op System: Pre-operative planning system.",
            "PostCare Recovery Tracker: Tracker for post-operative recovery.",
            "MediSupply Inventory System: Inventory management system for medical supplies.",
            "WellnessGuide Virtual Assistant: Virtual assistant for health and wellness."
        ]
    },
    {
        "company_name": "FashionForward",
        "company_description": f"""
        Industry: Fashion and Apparel
        Mission Statement: To bring the latest fashion trends to consumers with a focus on quality and style.
        Unique Selling Proposition (USP): Combines trendy designs with high-quality materials to create stylish and durable clothing.
        Target Market: Fashion-conscious individuals, trendsetters, and style enthusiasts.
        Product Offering: Trendy clothing, fashion accessories, designer handbags, footwear, and seasonal collections.
        Company Size: Mid-sized company with 700 employees and a global presence in both online and retail markets.
        History: Founded in 2014, FashionForward has quickly become a leading name in the fashion industry, known for its stylish designs and quality products.
        Target Audience: Fashion-conscious individuals, trendsetters, and style enthusiasts seeking the latest trends and high-quality clothing.
        """,
        "products": [
            "ChicMax Dresses: Trendy dresses for various occasions.",
            "UrbanEdge Jeans: Stylish jeans for urban wear.",
            "TrendSetter Handbags: Fashionable handbags.",
            "StyleFlex Sneakers: Sneakers designed for both style and comfort.",
            "GlamourWatch Collection: Collection of glamorous watches.",
            "ClassicFit Blazers: Classic blazers for professional wear.",
            "VintageCharm Accessories: Vintage-style accessories.",
            "SeasonalStyle Coats: Coats designed for seasonal fashion.",
            "ComfortWear T-Shirts: Comfortable t-shirts for everyday wear.",
            "NightOut Heels: High heels for night out events.",
            "Fashionista Sunglasses: Stylish sunglasses.",
            "CasualCool Shorts: Casual shorts for relaxed wear.",
            "EliteSport Activewear: Activewear for elite athletes.",
            "ElegantEvening Gowns: Elegant gowns for evening events.",
            "BusinessPro Suits: Professional suits for business settings.",
            "SummerBreeze Dresses: Dresses designed for summer wear.",
            "WinterWarm Scarves: Scarves for winter warmth.",
            "EcoFashion Tops: Eco-friendly tops.",
            "SpringBloom Skirts: Skirts designed for spring fashion.",
            "BoldStatement Jewelry: Statement jewelry pieces."
        ]
    }
]


In [11]:
companies_simplified = [
    {
        "company_name": "EcoTech Solutions",
        "company_description": "EcoTech Solutions provides eco-friendly technology products aimed at helping businesses and governments reduce their carbon footprint. Their offerings include solar panels, water filtration systems, and sustainable building materials. Founded in 2005, they are known for leading innovation in environmental technology.",
        "products": [
            "EcoClean Water Filtration System: Advanced filtration system for purifying water in homes and businesses.",
            "SolarMax Photovoltaic Panels: High-efficiency solar panels for generating renewable energy.",
            "GreenBuild Insulation Materials: Eco-friendly insulation materials for energy-efficient buildings.",
            "WasteAway Recycling Units: Innovative recycling units for managing waste sustainably.",
            "AgriTech Drip Irrigation System: Efficient irrigation system designed to conserve water in agriculture.",
            "WindGen Turbines: Wind turbines for harnessing wind energy.",
            "RainHarvest Collection Systems: Systems for collecting and utilizing rainwater.",
            "BioPack Biodegradable Packaging: Environmentally friendly packaging made from biodegradable materials.",
            "EVCharge Pro Stations: Electric vehicle charging stations for public and private use.",
            "HydroFlow Micro-Hydro Generators: Small-scale hydro generators for generating electricity from water flow.",
            "GeoTherm Heating Systems: Geothermal heating systems for sustainable temperature control.",
            "EcoSmart Thermostats: Smart thermostats that optimize energy usage.",
            "SolarRoof Shingles: Solar energy-generating roof shingles.",
            "GreenLight LED Lighting: Energy-efficient LED lighting solutions.",
            "EcoVentilation Systems: Ventilation systems that improve air quality and energy efficiency.",
            "EnviroHeat Pumps: Heat pumps that provide efficient heating and cooling.",
            "HydroPower Generators: Generators that produce electricity from hydro sources.",
            "EcoConserve Water Savers: Devices that reduce water consumption.",
            "BioWaste Digesters: Systems that convert organic waste into useful byproducts.",
            "SmartGrid Energy Management: Solutions for managing and optimizing energy distribution."
        ]
    },
    {
        "company_name": "HealthPro Pharmaceuticals",
        "company_description": "HealthPro Pharmaceuticals develops and sells a wide range of pharmaceutical products, including prescription drugs, over-the-counter medications, and vaccines. They are committed to improving health and well-being through rigorous research and development. Established in 1980, they are recognized for their long-standing reputation in the industry.",
        "products": [
            "CardioPro Blood Pressure Medication: Medication to manage high blood pressure.",
            "HealthPro Pain Relief Gel: Topical gel for relieving pain.",
            "ImmuneBoost Flu Vaccine: Vaccine to prevent influenza.",
            "GlucoCheck Blood Glucose Monitor: Device for monitoring blood glucose levels.",
            "NutriLife Multivitamins: Daily multivitamin supplement.",
            "Respira Inhaler: Inhaler for respiratory relief.",
            "NeuroPlus Cognitive Enhancer: Supplement to improve cognitive function.",
            "AllerFree Allergy Medication: Medication to relieve allergy symptoms.",
            "DiabeCare Insulin Pump: Insulin pump for diabetes management.",
            "ThermoPatch Heat Patches: Heat patches for pain relief.",
            "VitaBoost Vitamin D: Vitamin D supplement for bone health.",
            "OsteoGuard Bone Health: Supplement to support bone density.",
            "CoughCare Syrup: Syrup for cough relief.",
            "Dermacare Skin Ointment: Ointment for skin conditions.",
            "SleepWell Tablets: Tablets to aid sleep.",
            "HeartStrong Omega-3: Omega-3 supplement for heart health.",
            "FlexiJoint Glucosamine: Supplement for joint health.",
            "CalmEase Anti-Anxiety: Medication to reduce anxiety.",
            "FocusSharp Nootropic: Supplement to enhance mental focus.",
            "LiverHealth Detox: Supplement to support liver detoxification."
        ]
    },
    {
        "company_name": "TechNova Innovations",
        "company_description": "TechNova Innovations designs and manufactures consumer electronics like smart home devices, wearables, and high-performance laptops. Their focus is on creating cutting-edge technology that enhances everyday life. Since 2010, they have become a leading name in innovative and user-friendly electronics.",
        "products": [
            "HomeGuard Smart Security System: Advanced security system for smart homes.",
            "FitTrack Health Wearable: Wearable device for tracking fitness and health metrics.",
            "NovaTab Pro Tablet: High-performance tablet for work and entertainment.",
            "PowerBook Ultra Laptop: Powerful laptop for professional use.",
            "EchoBuds Wireless Earbuds: Wireless earbuds with superior sound quality.",
            "VistaVR Headset: Virtual reality headset for immersive experiences.",
            "SyncWatch Smartwatch: Smartwatch with fitness tracking and notifications.",
            "AuraSmart Light Bulbs: Smart light bulbs with adjustable brightness and colors.",
            "ChargeFast Wireless Charger: Fast wireless charger for multiple devices.",
            "SoundWave Bluetooth Speakers: High-quality Bluetooth speakers for home and travel.",
            "NanoScreen Protector: Durable screen protector for smartphones and tablets.",
            "AirPure Smart Air Purifier: Air purifier with smart controls and HEPA filters.",
            "SmartLock Door System: Smart locking system for enhanced home security.",
            "ThermoSense Smart Thermostat: Smart thermostat that learns and adapts to your schedule.",
            "RoboClean Vacuum: Robot vacuum cleaner with smart navigation.",
            "WaterSense Smart Faucet: Smart faucet that conserves water.",
            "CamGuard Security Camera: High-definition security camera with remote access.",
            "PowerHub Multi-Device Charger: Multi-device charger for simultaneous charging.",
            "EchoBeam Projector: High-definition projector for home theaters.",
            "SmartShade Automated Blinds: Automated blinds that adjust based on sunlight."
        ]
    },
    {
        "company_name": "PureBeauty Cosmetics",
        "company_description": "PureBeauty Cosmetics offers natural and eco-friendly beauty products, including skincare, makeup, and hair care. They emphasize using sustainable ingredients to enhance natural beauty. Founded in 2015, the company is dedicated to promoting sustainability in the beauty industry.",
        "products": [
            "RadiantGlow Face Cream: Face cream for radiant and hydrated skin.",
            "EcoLuxe Foundation: Eco-friendly foundation with natural ingredients.",
            "HerbalEssence Shampoo: Herbal shampoo for healthy hair.",
            "ClearSkin Acne Treatment: Acne treatment for clear skin.",
            "YouthRenew Anti-Aging Serum: Anti-aging serum for youthful skin.",
            "NaturalBalm Lip Care: Lip balm with natural ingredients.",
            "PureProtect Sunscreen: Sunscreen with broad-spectrum protection.",
            "Botanical Bliss Body Wash: Body wash with botanical extracts.",
            "SilkTouch Hair Conditioner: Hair conditioner for silky smooth hair.",
            "NatureFresh Deodorant: Natural deodorant without harmful chemicals.",
            "GreenTea Facial Cleanser: Facial cleanser with green tea extracts.",
            "VitaminC Brightening Serum: Serum with Vitamin C for brightening skin.",
            "HydraBoost Moisturizer: Moisturizer for intense hydration.",
            "SoothingAloe Gel: Aloe gel for soothing skin.",
            "Lavender Night Cream: Night cream with lavender for relaxation.",
            "RevitalEyes Eye Cream: Eye cream for reducing dark circles and puffiness.",
            "MineralRich Mud Mask: Mud mask with rich minerals for detoxifying skin.",
            "Organic Rosewater Toner: Toner with organic rosewater for refreshing skin.",
            "Chamomile Calming Mist: Calming mist with chamomile for soothing skin.",
            "Peppermint Foot Cream: Foot cream with peppermint for refreshing feet."
        ]
    },
    {
        "company_name": "EduFuture Learning",
        "company_description": "EduFuture Learning creates educational technology tools, such as e-learning platforms and virtual classrooms, to make learning more accessible and engaging. Their products are designed for students, teachers, and educational institutions. Since 2012, they have been a pioneer in integrating technology with education.",
        "products": [
            "LearnX E-learning Platform: Comprehensive platform for online learning.",
            "EduInteractive Whiteboard: Interactive whiteboard for engaging lessons.",
            "VirtualClass Pro: Virtual classroom software for remote teaching.",
            "MathMaster Educational Software: Software for mastering mathematics.",
            "E-Textbook Library: Digital library of textbooks for various subjects.",
            "QuizMaster Online Assessment: Online assessment tool for quizzes and exams.",
            "LangLearn Language App: Language learning app with interactive lessons.",
            "STEMExplorer Kit: Kit for exploring STEM concepts hands-on.",
            "VirtualLab Science Simulations: Simulations for virtual science experiments.",
            "AdaptiveLearn System: Adaptive learning system that personalizes education.",
            "HistoryHaven Digital Archive: Digital archive of historical documents and artifacts.",
            "GeoExplore Virtual Globe: Virtual globe for geography lessons.",
            "ChemLab Interactive Experiments: Interactive experiments for chemistry students.",
            "BioWorld Virtual Biology Lab: Virtual lab for biology experiments.",
            "PhysiCalc Physics Simulator: Physics simulator for understanding physical concepts.",
            "LiteratureLink Annotated Texts: Annotated texts for literature studies.",
            "ArtStudio Digital Art Tools: Digital tools for creating and studying art.",
            "MusicMaestro Learning Platform: Platform for learning and practicing music.",
            "EconEdu Financial Literacy Program: Program for teaching financial literacy.",
            "CoderPro Programming Courses: Courses for learning various programming languages."
        ]
    },
    {
        "company_name": "GreenHomes Real Estate",
        "company_description": "GreenHomes Real Estate specializes in building and selling eco-friendly, energy-efficient homes. Their properties are designed with sustainability in mind, catering to environmentally conscious buyers. Established in 2010, they are committed to setting new standards in eco-friendly construction.",
        "products": [
            "SolarVille Housing Project: Residential project with solar-powered homes.",
            "EcoPark Community Development: Sustainable community development.",
            "GreenTower Commercial Complex: Eco-friendly commercial office spaces.",
            "Sunshine Apartments: Energy-efficient apartment buildings.",
            "NatureHaven Villas: Villas designed with natural and sustainable materials.",
            "EcoLiving Townhouses: Townhouses with green building practices.",
            "GreenCity Condominiums: High-rise condominiums with sustainable features.",
            "Rainforest Estates: Luxury estates surrounded by preserved nature.",
            "EcoVillage Residential Area: Community with eco-friendly homes and amenities.",
            "SolarHeights Office Building: Office building powered by solar energy.",
            "ForestView Residences: Residences with views of forested areas.",
            "WindFarm Estates: Estates powered by wind energy.",
            "EcoLoft Urban Living: Urban living spaces with eco-friendly design.",
            "GreenBelt Communities: Communities with green spaces and sustainable housing.",
            "HydroHome Developments: Homes designed with water conservation features.",
            "BioDome Residences: Residences within biodome environments.",
            "SunBloom Gardens: Garden-style residences with sustainable features.",
            "RenewWave Apartments: Apartments with renewable energy systems.",
            "GeoLiving Complex: Complex designed with geothermal energy solutions.",
            "PureAir Estates: Estates with advanced air purification systems."
        ]
    },
    {
        "company_name": "FitLife Apparel",
        "company_description": "FitLife Apparel provides stylish and high-performance fitness clothing and accessories. Their products combine advanced fabric technology with trendy designs to support an active lifestyle. Founded in 2012, they have grown into a trusted brand in the fitness industry.",
        "products": [
            "FlexFit Leggings: Leggings designed for maximum flexibility and comfort.",
            "PowerCore Compression Shorts: Compression shorts for enhanced athletic performance.",
            "EcoYoga Mats: Eco-friendly yoga mats.",
            "RunFast Running Shoes: Running shoes designed for speed and comfort.",
            "LiftStrong Weightlifting Gloves: Gloves for safe and effective weightlifting.",
            "CoolDry T-Shirts: T-shirts with moisture-wicking technology.",
            "ThermoTrack Hoodies: Hoodies designed for temperature regulation.",
            "BalanceFit Sports Bras: Sports bras with excellent support.",
            "ProStretch Yoga Pants: Yoga pants with optimal stretch.",
            "Endurance Socks: Socks designed for endurance activities.",
            "QuickDry Swimwear: Swimwear that dries quickly.",
            "ReflectiveJog Safety Gear: Safety gear with reflective elements for night jogging.",
            "ActiveFlex Tracksuits: Tracksuits designed for active lifestyles.",
            "UltraLight Jackets: Lightweight jackets for outdoor activities.",
            "CoreStrength Athletic Belts: Athletic belts for core support.",
            "SpeedGrip Training Shoes: Training shoes with superior grip.",
            "StretchTech Resistance Bands: Resistance bands for strength training.",
            "HydroFlow Water Bottles: Water bottles designed for hydration on the go.",
            "PulseMonitor Fitness Tracker: Fitness tracker for monitoring vital signs.",
            "ZenBalance Meditation Cushions: Cushions designed for meditation practice."
        ]
    },
    {
        "company_name": "Gourmet Delights",
        "company_description": "Gourmet Delights offers high-quality, gourmet food products such as artisanal cheeses, chocolates, and specialty oils. Their focus is on delivering exceptional taste and quality for food enthusiasts and gourmet chefs. Established in 2015, they have quickly gained a reputation for excellence in the food industry.",
        "products": [
            "Truffle Infused Olive Oil: Olive oil infused with rich truffle flavor.",
            "Gourmet Dark Chocolate Bars: High-quality dark chocolate bars.",
            "Artisanal Cheddar Cheese: Handcrafted cheddar cheese.",
            "Organic Spicy Nuts: Organic nuts with a spicy twist.",
            "Premium Vanilla Extract: High-quality vanilla extract.",
            "Heritage Breed Pork Sausages: Sausages made from heritage breed pork.",
            "Handmade Belgian Waffles: Traditional Belgian waffles.",
            "Wildflower Honey: Pure honey from wildflowers.",
            "Balsamic Vinegar of Modena: Authentic balsamic vinegar.",
            "Mediterranean Sea Salt: Sea salt from the Mediterranean.",
            "French Brie Cheese: Creamy French brie cheese.",
            "Italian Espresso Beans: Premium Italian espresso beans.",
            "Maple Glazed Salmon: Salmon glazed with maple syrup.",
            "Black Truffle Butter: Butter infused with black truffle.",
            "Gourmet Popcorn Variety: Assorted gourmet popcorn flavors.",
            "Smoked Almonds: Almonds with a smoky flavor.",
            "Artisan Sourdough Bread: Handcrafted sourdough bread.",
            "Spiced Chai Tea: Chai tea with a blend of spices.",
            "Exotic Fruit Jam: Jam made from exotic fruits.",
            "Organic Pasta Selection: Assorted organic pasta varieties."
        ]
    },
    {
        "company_name": "GreenBytes Tech",
        "company_description": "GreenBytes Tech develops software solutions that help businesses manage energy use and reduce their environmental impact. Their tools include energy management systems and carbon footprint calculators. Founded in 2010, they are dedicated to helping organizations achieve their sustainability goals.",
        "products": [
            "EcoManage Energy Management Software: Software for managing energy usage.",
            "SustainReport Sustainability Reporting Tools: Tools for sustainability reporting.",
            "CarbonCalc Footprint Calculator: Calculator for measuring carbon footprint.",
            "GreenChain Supply Management: Software for green supply chain management.",
            "EcoProject Project Management Tool: Project management software for eco-projects.",
            "ResourceSaver Efficiency Suite: Suite of tools for resource efficiency.",
            "GreenAudit Compliance Checker: Software for checking environmental compliance.",
            "EnviroTracker Impact Monitoring: Monitoring tool for environmental impact.",
            "EcoPlan Strategic Planning Software: Software for strategic environmental planning.",
            "SustainDash Analytics Dashboard: Dashboard for sustainability analytics.",
            "EnergyWatch Consumption Monitor: Monitor for tracking energy consumption.",
            "RenewTrack Renewable Integration: Tool for integrating renewable energy.",
            "WasteReduce Waste Management: Software for managing waste reduction.",
            "CarbonMap Emission Mapper: Mapping tool for carbon emissions.",
            "EcoFleet Fleet Management: Management tool for eco-friendly fleets.",
            "WaterWise Usage Tracker: Tracker for water usage.",
            "GreenBudget Financial Planning: Financial planning tool for green projects.",
            "RecycleSmart Waste Optimization: Tool for optimizing waste recycling.",
            "EcoScore Performance Metrics: Metrics tool for environmental performance.",
            "EnviroSync Integration Platform: Platform for integrating environmental data."
        ]
    },
    {
        "company_name": "PureSound Audio",
        "company_description": "PureSound Audio creates high-quality audio equipment, including speakers, headphones, and audio players. Their products are designed for audiophiles and professionals who demand superior sound quality. Since 2012, they have become a respected name in the audio equipment industry.",
        "products": [
            "PureBass High-Fidelity Speakers: Speakers with high-fidelity sound quality.",
            "NoiseShield Pro Headphones: Headphones with advanced noise-cancelling technology.",
            "StudioMaster Monitors: Studio monitors for professional audio production.",
            "PocketSound Bluetooth Speaker: Portable Bluetooth speaker.",
            "HiRes Audio Player: High-resolution audio player.",
            "ClearVoice Microphone: Microphone with clear sound capture.",
            "BassBoost Subwoofer: Subwoofer with enhanced bass.",
            "WirelessSound Home System: Wireless home audio system.",
            "SoundSync Wireless Transmitter: Wireless transmitter for audio devices.",
            "PureTone Earbuds: Earbuds with pure sound quality.",
            "AcousticPro Sound Panels: Sound panels for acoustic treatment.",
            "ConcertMaster Amplifier: Amplifier for live performances.",
            "StudioConnect Audio Interface: Audio interface for recording studios.",
            "SoundBoost Earplugs: Earplugs for sound protection.",
            "PureBass Wireless Earbuds: Wireless earbuds with pure bass.",
            "NoiseCancel Over-Ear Headphones: Over-ear headphones with noise-cancelling technology.",
            "LiveStream Microphone: Microphone for live streaming.",
            "HomeTheater Surround System: Surround sound system for home theaters.",
            "ProMix Audio Mixer: Audio mixer for professional use.",
            "QuietComfort Noise Cancelling Headphones: Comfortable headphones with noise-cancelling features."
        ]
    },
    {
        "company_name": "HomeHaven Furnishings",
        "company_description": "HomeHaven Furnishings sells stylish and affordable furniture for homes, including living room sets, bedroom furniture, and home décor. Their designs cater to both modern and classic tastes. Founded in 2013, they have become a trusted name in the home furnishings market.",
        "products": [
            "ComfortPlus Sofa: Sofa designed for maximum comfort.",
            "Elegance Dining Table: Elegant dining table for stylish dining rooms.",
            "Modern Loft Bed Frame: Bed frame with a modern loft design.",
            "OfficePro Desk: Desk designed for professional use.",
            "Luxury Recliner Chair: Recliner chair with luxurious comfort.",
            "Urban Chic Coffee Table: Stylish coffee table for urban homes.",
            "Classic Wardrobe: Classic design wardrobe for ample storage.",
            "Designer Lamp Collection: Collection of designer lamps.",
            "EcoWood Bookshelves: Bookshelves made from eco-friendly wood.",
            "Premium Mattress Set: High-quality mattress set for ultimate comfort.",
            "VelvetLuxe Armchair: Velvet armchair for a luxurious feel.",
            "SmartHome Entertainment Unit: Entertainment unit with smart features.",
            "RusticKitchen Island: Rustic-style kitchen island.",
            "Vintage Mirror Set: Set of vintage mirrors.",
            "SpaceSaver Shoe Rack: Shoe rack designed to save space.",
            "DualComfort Mattress: Mattress with dual comfort zones.",
            "Heritage Dining Set: Traditional dining set with heritage design.",
            "RelaxZone Bean Bag: Comfortable bean bag chair.",
            "Craftsman Workbench: Workbench designed for craftsmen.",
            "Timeless Bedside Table: Classic bedside table design."
        ]
    },
    {
        "company_name": "BioLife Supplements",
        "company_description": "BioLife Supplements produces natural health supplements, including vitamins, probiotics, and protein powders. Their products are aimed at supporting a healthy lifestyle with high-quality natural ingredients. Founded in 2015, they have established themselves as a trusted provider of natural health products.",
        "products": [
            "BioLife Multivitamin: Daily multivitamin supplement.",
            "ProBio Advanced Probiotics: Probiotics for digestive health.",
            "HerbalBoost Energy Capsules: Energy capsules with herbal ingredients.",
            "VitaC Immune Support: Vitamin C supplement for immune support.",
            "PureOmega Fish Oil: Fish oil supplement for heart health.",
            "PlantPower Protein Powder: Plant-based protein powder.",
            "JointFlex Glucosamine: Glucosamine supplement for joint health.",
            "DigestEase Enzyme Blend: Enzyme blend for digestive health.",
            "CalMag Calcium Magnesium: Calcium and magnesium supplement.",
            "SleepWell Melatonin: Melatonin supplement for sleep support.",
            "LiverCare Detox: Detox supplement for liver health.",
            "VisionGuard Lutein: Lutein supplement for eye health.",
            "HeartHealth CoQ10: CoQ10 supplement for heart health.",
            "StressRelief Ashwagandha: Ashwagandha supplement for stress relief.",
            "SkinGlow Collagen: Collagen supplement for skin health.",
            "MemorySharp Ginkgo Biloba: Ginkgo Biloba supplement for cognitive health.",
            "MuscleMax BCAA: BCAA supplement for muscle recovery.",
            "EnergyPlus Iron Supplement: Iron supplement for energy support.",
            "BoneStrength Calcium: Calcium supplement for bone health.",
            "AntiInflame Turmeric: Turmeric supplement for anti-inflammatory support."
        ]
    },
    {
        "company_name": "UrbanEats Food Truck",
        "company_description": "UrbanEats Food Truck serves gourmet street food, such as burgers, tacos, and salads, from a fleet of food trucks. They use fresh, locally sourced ingredients to create delicious and convenient meals. Established in 2018, they have quickly become a favorite among city dwellers.",
        "products": [
            "Gourmet Angus Burger: High-quality Angus beef burger.",
            "Truffle Grilled Cheese: Grilled cheese sandwich with truffle flavor.",
            "Quinoa Power Salad: Salad with quinoa and fresh vegetables.",
            "BBQ Pulled Pork Sandwich: Sandwich with BBQ pulled pork.",
            "Spicy Fish Tacos: Tacos with spicy fish filling.",
            "Veggie Delight Wrap: Wrap with assorted vegetables.",
            "Lemonade Fresh: Freshly made lemonade.",
            "Craft Root Beer: Craft-brewed root beer.",
            "Loaded Nachos: Nachos loaded with toppings.",
            "Street Corn Elote: Mexican-style street corn.",
            "Buffalo Chicken Wings: Chicken wings with buffalo sauce.",
            "Gourmet Mac and Cheese: Mac and cheese with gourmet ingredients.",
            "Chipotle Steak Burrito: Burrito with chipotle steak.",
            "Hawaiian Poke Bowl: Poke bowl with Hawaiian flavors.",
            "Sweet Potato Fries: Fries made from sweet potatoes.",
            "Falafel Wrap: Wrap with falafel and fresh vegetables.",
            "Artisan Pizza Slices: Slices of artisan pizza.",
            "Hand-Crafted Ice Cream: Handmade ice cream.",
            "Sriracha Chicken Sandwich: Chicken sandwich with sriracha sauce.",
            "Grilled Veggie Skewers: Skewers with grilled vegetables."
        ]
    },
    {
        "company_name": "AdventureGear Co.",
        "company_description": "AdventureGear Co. sells durable and innovative outdoor gear, including hiking boots, camping tents, and climbing equipment. Their products are designed for adventure enthusiasts who need reliable gear for tough conditions. Founded in 2010, they are known for their rugged and high-quality outdoor equipment.",
        "products": [
            "TrekMaster Hiking Boots: Durable hiking boots for all terrains.",
            "EcoTent Camping Tent: Eco-friendly camping tent.",
            "ClimbSafe Harness: Safety harness for climbing.",
            "AquaPro Kayak: Professional-grade kayak.",
            "TrailBlaze Backpack: Backpack designed for hiking.",
            "ExtremeWeather Sleeping Bag: Sleeping bag for extreme weather conditions.",
            "MountainPeak Trekking Poles: Trekking poles for mountain hiking.",
            "RapidInflate Sleeping Pad: Inflatable sleeping pad.",
            "HydroFlow Water Filter: Water filter for outdoor use.",
            "Survivor Multi-Tool: Multi-tool with various functions.",
            "SolarCharge Portable Panel: Portable solar panel for charging devices.",
            "WindResist Jacket: Jacket designed to resist wind.",
            "FireStarter Kit: Kit for starting fires.",
            "GeoCompass Navigation Device: Navigation device for geolocation.",
            "RockClimber Chalk Bag: Bag for climbing chalk.",
            "StormGuard Rain Gear: Rain gear for protection against storms.",
            "TrailMaster GPS: GPS device for trail navigation.",
            "CampCook Portable Stove: Portable stove for camping.",
            "ThermoPack Insulated Bottle: Insulated bottle for keeping drinks hot or cold.",
            "DrySafe Waterproof Bag: Waterproof bag for protecting gear."
        ]
    },
    {
        "company_name": "TechHealth Solutions",
        "company_description": "TechHealth Solutions provides advanced health technology products like telehealth platforms and patient monitoring systems. Their solutions are designed to improve patient care and streamline healthcare processes. Since 2012, they have been dedicated to revolutionizing healthcare through technology.",
        "products": [
            "HealthManage Software: Software for managing healthcare services.",
            "TeleCare Telehealth Platform: Platform for telehealth services.",
            "VitalsMonitor Patient System: Patient monitoring system.",
            "EHRPlus Electronic Health Records: Electronic health records system.",
            "MediData Analytics: Analytics tool for medical data.",
            "CareConnect Communication System: Communication system for healthcare providers.",
            "RemoteTrack Monitoring Devices: Devices for remote patient monitoring.",
            "SmartAlert Emergency Response: Emergency response system.",
            "HealthPortal Patient Interface: Interface for patient engagement.",
            "BioScan Diagnostic Tools: Diagnostic tools for healthcare professionals.",
            "MediTrack Wearable: Wearable device for tracking health metrics.",
            "DocLink Physician Network: Network for physician collaboration.",
            "ClinicFlow Management: Management software for clinics.",
            "PatientConnect App: App for patient communication and management.",
            "HealthSync Integration: Integration platform for health systems.",
            "PharmaCheck Drug Monitoring: Drug monitoring system.",
            "SurgiPlan Pre-Op System: Pre-operative planning system.",
            "PostCare Recovery Tracker: Tracker for post-operative recovery.",
            "MediSupply Inventory System: Inventory management system for medical supplies.",
            "WellnessGuide Virtual Assistant: Virtual assistant for health and wellness."
        ]
    },
    {
        "company_name": "FashionForward",
        "company_description": "FashionForward offers trendy, high-quality clothing and accessories for fashion-conscious individuals. Their products range from dresses and jeans to handbags and footwear, combining style with durability. Founded in 2014, they have become a leading name in the global fashion industry.",
        "products": [
            "ChicMax Dresses: Trendy dresses for various occasions.",
            "UrbanEdge Jeans: Stylish jeans for urban wear.",
            "TrendSetter Handbags: Fashionable handbags.",
            "StyleFlex Sneakers: Sneakers designed for both style and comfort.",
            "GlamourWatch Collection: Collection of glamorous watches.",
            "ClassicFit Blazers: Classic blazers for professional wear.",
            "VintageCharm Accessories: Vintage-style accessories.",
            "SeasonalStyle Coats: Coats designed for seasonal fashion.",
            "ComfortWear T-Shirts: Comfortable t-shirts for everyday wear.",
            "NightOut Heels: High heels for night out events.",
            "Fashionista Sunglasses: Stylish sunglasses.",
            "CasualCool Shorts: Casual shorts for relaxed wear.",
            "EliteSport Activewear: Activewear for elite athletes.",
            "ElegantEvening Gowns: Elegant gowns for evening events.",
            "BusinessPro Suits: Professional suits for business settings.",
            "SummerBreeze Dresses: Dresses designed for summer wear.",
            "WinterWarm Scarves: Scarves for winter warmth.",
            "EcoFashion Tops: Eco-friendly tops.",
            "SpringBloom Skirts: Skirts designed for spring fashion.",
            "BoldStatement Jewelry: Statement jewelry pieces."
        ]
    }
]


In [12]:
writing_tones = [
    "Professional",
    "Enthusiastic",
    "Witty",
    "Inspirational",
    "Trustworthy"
]

In [13]:
finetune_dataset_dict = {}
user_prompts = []
system_prompts = []
post_tones = []
post_types = []
post_lengths = []

### Generate Synthetic Dataset

In [14]:
sales_prompts = []
for company in companies:
  for product in company['products']:
    for post in short_posts + medium_posts + long_posts:
      for tone in writing_tones:
        post_type = post["type"]
        length = post["length"]
        complete_criteria = f"""
        {universal_criteria}
        {post['criteria']}
        """
        prompt = f"""
        You are an expert sales copywriter for social media that specializes in modern sales and the "attention economy".

        Your task is to a social media post for sales with the following characteristics:
        1. Company name: {company['company_name']}
        2. Company Description: {company['company_description']}
        3. Product: {product}
        4. Post length: {post["length"]}
        5. Post type: {post["type"]}
        6. Post characterisitics: {complete_criteria}
        7. Writing tone: {tone}

        Your output MUST be ONLY the social media post with NO additional information, text, commentary or anything extra.
        """

        prompt_metadata = {
            "company_name" : company['company_name'],
            "company_description" : {company['company_description']},
            "product" : product,
            "post_characterisitics": complete_criteria,
            "post_type" : post['type'],
            "length" : post['length'],
            "tone" : tone
        }
        sales_prompts.append((prompt,prompt_metadata))

In [15]:
print(len(sales_prompts))

17600


**The code commented out below works but is commented out to prevent accidental running and overwriting existing dataset.**

In [17]:
# finetune_dataset_path = '/content/drive/My Drive/synthetic_dataset/sales_dataset/finetune_dataset.csv'

In [19]:
# batch_size = 400

In [20]:
# batches = [sales_prompts[i:i + batch_size] for i in range(0, len(sales_prompts), batch_size)]

In [21]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# metadata_df = pd.DataFrame(columns=['batch_index', 'run_time'])
# metadata_csv_path = '/content/drive/My Drive/synthetic_dataset/sales_dataset/test_metadata.csv'
# metadata_df.to_csv(metadata_csv_path, index=False)

In [23]:
# finetune_dataset_df = pd.DataFrame(columns=['user_prompt', 'response', 'post_tone', 'post_type', 'post_length'])
# finetune_dataset_df.to_csv(finetune_dataset_path, index=False)

In [27]:
# for idx, batch in enumerate(batches):
#   start_time = time.time()
#   prompt_responses_dict = ask_llm(batch, groq_client, llama8b)
#   end_time = time.time()
#   run_time = round(end_time - start_time,4)

#   batch_df = pd.DataFrame(prompt_responses_dict)
#   batch_df.to_csv(finetune_dataset_path, mode='a', index=False, header=False)

#   metadata = {
#       'batch_index':idx,
#       'run_time': run_time
#   }

#   metadata_df = pd.DataFrame([metadata])
#   metadata_df.to_csv(metadata_csv_path, mode='a', index=False, header=False)

In [28]:
# del finetune_dataset_df

### Create Prompts For Baseline Responses

The prompts used to generate the fine tuned responses specify the criteria for the responses. The prompts used to generate the baseline responses do not have this criteria in the prompt since the goal of the fine tuned model is to satisfy this criteria without explicitly being asked too.

The code below loads the finetune dataset created above and overwrites the user prompt with the correct (non finetune) user prompts that will be used to generate baseline responses in another notebook.

In [29]:
# finetune_dataset_df = pd.read_csv(finetune_dataset_path)

In [31]:
# user_prompts = []
# system_prompts = []
# for company in companies_simplified:
#   for product in company['products']:
#     for post in short_posts + medium_posts + long_posts:
#       for tone in writing_tones:
#         post_type = post["type"]
#         length = post["length"]


#         user_prompt = f"""Write me a sales post for my company's product with the following characteristics:
# 1. company description: {company['company_description']}
# 2. product: {product}
# 3. post length: {length}
# 4. post type: {post_type}
# 5. post tone: {tone}

# Your output MUST be ONLY the social media post with NO additional information, text, commentary or anything extra."""

#         system_prompts.append("You are an expert sales copywriting assistant for social media who understands the attention economy and modern digital advertisment.")
#         user_prompts.append(user_prompt)

# finetune_dataset_df["system_prompt"] = system_prompts
# finetune_dataset_df["user_prompt"] = user_prompts

In [32]:
# finetune_dataset_df.to_csv(finetune_dataset_path, index = False)